In [1]:
import random
import torch
SEED = 2222
random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
import nltk
from vncorenlp import VnCoreNLP
nltk.download('punkt')
annotator = VnCoreNLP("VnCoreNLP-master\VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\16262\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from iteration_utilities import deepflatten
def tokenize_en(text):
    return [tok.text for tok in nltk.word_tokenize(text)]
def tokenize_vi(text):
    return [tok for tok in deepflatten(annotator.tokenize(text), depth=1)]

In [4]:
import pandas as pd
from nlp import load_dataset


# Define the fields for your data
fields = {"English": ("src", tokenize_en), "Vietnamese": ("trg", tokenize_vi)}

def create_raw_dataset():
    data_dir = ""
    en_sents = open(data_dir + 'english.txt', "r",encoding="utf-8" ).read().splitlines()
    vi_sents = open(data_dir + 'vietnamese.txt', "r" ,encoding="utf-8").read().splitlines()
    return {
        "English": [line for line in en_sents[:5000]],
        "Vietnamese": [line for line in vi_sents[:5000]],
    }
raw_data = create_raw_dataset()

from sklearn.model_selection import train_test_split

df = pd.DataFrame(raw_data, columns=["English", "Vietnamese"])
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.125)

train.to_json("train.json", orient="records", lines=True)
test.to_json("test.json", orient="records", lines=True)
val.to_json("val.json", orient="records", lines=True)

# Load the data from the json files
train_data = load_dataset("json", data_files="train.json", split="train", field=fields)
test_data = load_dataset("json", data_files="test.json", split="test", field=fields)
val_data = load_dataset("json", data_files="val.json", split="val", field=fields)


TypeError: 'bytes' object cannot be interpreted as an integer